In [1]:
import numpy as np 
import h5py

h=0.704

/Users/michaelkatz/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Manual Filtering

This script allows the user to manually filter for some ID changes that were missed by the algorithm. This number usually numbers under 100. The idea is to find clear instances where the first time an ID is seen is for a large black hole. This clearly means a seed merged into the large black hole and the seed ID was chosen. 

1) The user can look at the large black holes whose IDs appear randomly

2) Check the last snapshot for black holes with an almost identical mass

3) Add them to the list

4) Make the necessary changes for these IDs

This method is not perfect. But close examination shows it will solve a majority of the remaining issues. If you want to skip this execute the next cell.

In [ ]:
##### ONLY EXECUTE THIS IF YOU DO NOT WANT TO DO THE MANUAL SEARCH ####
with open('manual_search_not_done.txt', 'r') as fp:
    fp.write('not doing manual search')

#### First, load information from the new details catalog. This step takes the longest due to the sorting operation. 

In [ ]:
with h5py.File('bhs_details_new.hdf5', 'r') as f:
    pid_det_old = f['id'][:] 
    pid_det = f['id_new'][:] 
    time_det = f['time'][:]
    mass_det = f['mass'][:]*1e10/h

details =  np.core.records.fromarrays([f['id'][:], f['mass'][:]*1e10/0.704, f['time'][:]], names='id,mass,time')

sort_inds = np.argsort(details, order=('time', 'mass'))

details = details[sort_inds]

#sort old ids in same order as the new ideas
pid_det_old = pid_det_old[sort_inds]

#do the same for index into the dataset
old_indices = np.arange(len(pid_det_old))[sort_inds]

#### Sort information for the all bhs dataset and sort the data

In [1]:
with h5py.File('bhs_all_new.hdf5', 'r') as f:
    pid = f['ParticleIDs_new'][:] 
    pid_old = f['ParticleIDs'][:] 
    snap = f['Snapshot'][:]
    mass = f['BH_Mass'][:]

checker = [(pid[i], snap[i], mass[i], pid_old[i]) for i in range(len(pid))]

checker = np.asarray(checker, dtype=[('id', np.dtype(np.uint64)), ('snap', np.dtype(float)), ('mass', np.dtype(float)), ('id_old', np.dtype(float))])

checker = np.sort(checker, order=['id', 'snap'])

/Users/michaelkatz/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


We need to find all the black holes who appeared at a higher mass than the seeds for its first appearance. We also need to find the last time a bh appeared and its mass. Therefore, we can find bhs that disappeared right before a snapshot where a new black hole appeared with almost exactly the same mass as the one that disappeared. 

In [ ]:
#find the first instance an id appears in the all dataset
pid_uni, uni_ind = np.unique(checker['id'], return_index=True)

#this is the mass when it first appears
init_mass = checker['mass'][uni_ind]

#need to check indices where the first time seeing a black hole, it has a mass greater than 10^6
inds_check = np.where(init_mass>1e6)[0]

#snaps that require you to check black holes in
snaps_init_to_check = checker['snap'][uni_ind][inds_check]

#new ids that need to be checked
ids_to_check = checker['id'][uni_ind][inds_check]

#masses of those to check
mass_to_check = checker['mass'][uni_ind][inds_check]

#old ids 
old_ids_to_check = checker['id_old'][uni_ind][inds_check]

#refers the ids because now you want to see the last time a black hole appears
pid_uni_backward, last_ind = np.unique(checker['id'][::-1], return_index=True)

#mass at bh's final appearance
final_mass = checker['mass'][::-1][last_ind]
snaps_final = checker['snap'][::-1][last_ind]

print('We need to check %i black holes.'%len(ids_to_check))

Below, we search for all the black holes we found in the earlier steps. We put them in an array that will let us observe if these misplaced black holes can be fixed.

In [4]:
out = []
finished=0
print(len(ids_to_check))

#we have the information for the black hole that is left over. Now we search
#for the second black hole that lost its id in the merger that fits criteria to 
#simplify search.
for id1, mass, snap, id1_old in np.array([ids_to_check, mass_to_check, snaps_init_to_check, old_ids_to_check]).T:
    
    #find where the details id is the id of black hole left over and take its time and mass values
    inds = np.where(details['id'] == id1)[0]
    time_sort = details['time'][inds]
    mass_sort = details['mass'][inds]

    #find where the log10 of the mass values takes the largest jump (upping mass orders of magnitude)
    ind_switch  = np.where(np.diff(np.log10(details['mass'][inds])) == np.diff(np.log10(details['mass'][inds])).max())[0][0]

    #ind_out_1 is when they switch
    ind_out_1 = inds[ind_switch]
    
    #ind_out_2 is first index after switch
    ind_out_2 = inds[ind_switch+1]

    time_switch = details['time'][ind_out_1]

    time_out_2 = details['time'][ind_out_2]

    mass_switch = details['mass'][ind_out_1]

    #looking for other blackholes that have the same specific time value of the switch
    inds_at_switch = np.where(details['time'] == time_switch)[0]
    
    #look at the first black hole found above
    ind_search = inds_at_switch[-1] + 1

    inds_of_interest = []
    
    #walk through black holes for those that fit these criterion and append them to ids_of_interest to look at later
    while details['time'][ind_search]<details['time'][ind_out_2]:
        if details['mass'][ind_search]>details['mass'][ind_out_2]/2 and details['mass'][ind_search]<details['mass'][ind_out_2]:
            inds_of_interest.append(ind_search)
        ind_search += 1
    
    #look at the old ids of interes
    ids_of_interest = pid_det_old[inds_of_interest]

    for i, id2 in enumerate(ids_of_interest):
        #find everywhere this id of interest appears in old details ids
        inds_find = np.where((pid_det_old == id2))[0]

        if len(inds_find) != 0:
            total mass of both black holes before switch
            total_mass = details['mass'][inds_find][-1] + details['mass'][ind_out_1]
            #if this pair of black holes have the same time switch coordinate and 
            #the mass of the remaing black hole is greater than or equal to
            #the combination of the constituent black holes. Also the total mass 
            #is greater than half of the mass of black hole that loses its id (this constrains output).
            if details['time'][inds_find][-2] == time_switch and total_mass <= details['mass'][ind_out_2] and total_mass > details['mass'][ind_out_2]/2:
                ind_in = inds_find[-1]
                
                #append black holes that fit this criterion
                out.append([finished, old_indices[ind_out_1], old_indices[ind_in], old_indices[ind_out_2], pid_det_old[ind_out_1], pid_det_old[ind_in], pid_det_old[ind_out_2], details['mass'][ind_out_1], details['mass'][ind_in], details['mass'][ind_out_2],details['time'][ind_out_1], details['time'][ind_in], details['time'][ind_out_2]])


91
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91


Now we output this array we found, look through it and locate mergers that we missed so we can correct their ids.

In [17]:
outc = np.asarray(out, dtype=object)
for j, out_vals in enumerate(outc):
    #here, j is the number of pair we searched, out_vals[0] is the merger number. 
    #then there is the time (scale) at which this takes place. 
    #Finally, the mass of the larger black hole that has its id removed and the mass of the black hole
    #appears in the next snapshot with a very similar mass. This black hole id belonged to a much smaller 
    #black hole at the previous snapshot. 
    #some mergers may have more than one potential pair. This is why we search manually. 
    print(j, out_vals[0], out_vals[7:10]/1e6)

0 0 [0.142123578125 33.742188 33.891904]
1 0 [0.142123578125 17.498154 33.891904]
2 1 [0.142058234375 1076.441856 1080.282624]
3 2 [0.142045453125 4394.971648 4395.127808]
4 3 [0.154813921875 215.23152 216.9716]
5 4 [0.142326703125 2632.229888 2633.338112]
6 5 [0.1445838125 667.774208 668.646336]
7 6 [0.142045453125 2896.420352 2897.045504]
8 7 [0.142045453125 4430.951424 4433.86368]
9 8 [0.142051140625 1905.653504 1905.965824]
10 8 [0.142051140625 1567.471616 1905.965824]
11 9 [0.143342328125 283.428992 283.669024]
12 10 [0.14320028125 304.981536 305.375008]
13 11 [0.143379265625 6653.906432 6654.247424]
14 12 [0.142345171875 37.43608 37.578976]
15 12 [0.142345171875 28.287358 37.578976]
16 12 [0.142345171875 24.935512 37.578976]
17 13 [0.142046875 1287.933312 1288.683136]
18 14 [0.1420525625 5657.017344 5657.201664]
19 15 [0.142613640625 2957.641984 2958.068224]
20 16 [0.142045453125 276.880704 311.573856]
21 16 [0.142045453125 311.421888 311.573856]
22 16 [0.142045453125 202.572448 

Look through the list above for pairs that do not look like reasonable candidates. For example, if we have:

0 0 [0.142123578125 33.742188 33.891904]

1 0 [0.142123578125 17.498154 33.891904]

These are both for the same merger 0 (this is just the number merger in the mergers we are examining, not the overall index). It is clear the first entry is the black hole we are looking for. The second is not. Therefore we add the index 1 to the bad list below to remove it from the final tally.

Another example:

41 34 [0.142045453125 1499.985792 1500.79552]

42 34 [0.142045453125 913.122176 1500.79552]

43 34 [0.142045453125 1383.347968 1500.79552]

All for the same merger. For what we are looking for, a seed black hole into a very large black hole, we expect the mass change in the $\geq10^{10}M_\odot$ black to be about equal to the seed mass. Therefore, the correct merger in this example is 41. 

42 is clearly wrong. 43 has too large of a change to be from a seed mass black hole. We add 42 and 43 to the bad list.

In [ ]:
bad_list = [1, 15, 16, 20, 22, 27, 30, 31, 42, 43, 47, 50, 53, 55, 10, 36, 52, 61, 71]

In [27]:
#remove the bad list
out_cleaned = np.delete(outc, , axis=0)

In [28]:
print(out_cleaned[:, 7:10])

[[142123.58 33742188.0 33891904.0]
 [142058.23 1076441900.0 1080282600.0]
 [142045.45 4394971600.0 4395128000.0]
 [154813.92 215231520.0 216971600.0]
 [142326.7 2632230000.0 2633338000.0]
 [144583.81 667774200.0 668646340.0]
 [142045.45 2896420400.0 2897045500.0]
 [142045.45 4430951400.0 4433863700.0]
 [142051.14 1905653500.0 1905965800.0]
 [143342.33 283429000.0 283669020.0]
 [143200.28 304981540.0 305375000.0]
 [143379.27 6653906400.0 6654247400.0]
 [142345.17 37436080.0 37578976.0]
 [142046.88 1287933300.0 1288683100.0]
 [142052.56 5657017300.0 5657201700.0]
 [142613.64 2957642000.0 2958068200.0]
 [142045.45 311421900.0 311573860.0]
 [143428.98 1511420400.0 1511676200.0]
 [142048.3 6158892500.0 6161889300.0]
 [142090.9 1193169000.0 1193512700.0]
 [142423.3 2046718700.0 2046874900.0]
 [142399.16 50090200.0 50266760.0]
 [142045.45 7573097000.0 7574219000.0]
 [142045.45 1590610700.0 1592244400.0]
 [142045.45 85767470.0 85909944.0]
 [142045.45 1681008500.0 1681164800.0]
 [142045.45 4066

# REMEMBER TO ADD DIRECTORY

In [ ]:
directory = './extraction_files'

In [30]:
np.savetxt(directory + 'fix_list.txt', out_cleaned, fmt = '%i\t%i\t%i\t%i\t%i\t%i\t%i\t%.18e\t%.18e\t%.18e\t%.18e\t%.18e\t%.18e', header = 'search_index\tind_out_1\tind_in\tind_out_2\tid_out_1\tid_in\tid_out2\tmass_out_1\tmass_in\tmass_out_2\ttime_out_1\ttime_in\ttime_out_2')